# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [51]:
import os
from glob import glob

# Write your code below.
print(os.getenv('PRICE_DATA'))
price_files = glob(os.path.join(os.getenv('PRICE_DATA'), '**/*.parquet'), recursive=True)
price_files

../../05_src/data/prices/


['../../05_src/data/prices/NXC/NXC_2002/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_2002/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_2005/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_2005/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_2004/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_2004/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_2003/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_2003/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_1997/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_1997/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_1999/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_1999/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_1998/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_1998/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_1996/part.0.parquet',
 '../../05_src/data/prices/NXC/NXC_1996/part.1.parquet',
 '../../05_src/data/prices/NXC/NXC_2010/part.0.parquet',
 '../../05_src/data/prices/NXC/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [54]:
# Write your code below.
dd_pp = dd.read_parquet(price_files,set_index='ticker')

dd_shift = dd_pp.groupby('ticker',group_keys=False).apply(lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                                                                             Adj_Close_lag = x['Adj Close'].shift(1)))

dd_feat = dd_shift.assign(returns = lambda x: x['Close']/x['Close_lag_1']-1, hi_lo_range = lambda x: x['High'] - x['Low'])
dd_feat

/var/folders/xd/827g3ndd0kg_81hlxd5mbp2m0000gn/T/ipykernel_69607/4090045487.py:4: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  dd_shift = dd_pp.groupby('ticker',group_keys=False).apply(lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag,returns,hi_lo_range
npartitions=2893,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,string,string,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
pd_pf = dd_feat.compute()

# To avoid repeating the above relatively expensive computing when I tested the following code, 
# I moved the code to the next cell.

In [ ]:
# Zhen's notes: the dataset has been grouped by 'ticker' and by observation, within each ticker group, the data is sorted ascending by Date.
# To ensure sorting by date ascending so that moving average is meaningful, I use the following sort_values() function.
df_sorted = pd_pf.groupby('ticker').apply(lambda x: x.sort_values('Date'))
df_sorted
pd_pf['moving_avg_returns'] = pd_pf['returns'].rolling(10).mean()
pd_pf

/var/folders/xd/827g3ndd0kg_81hlxd5mbp2m0000gn/T/ipykernel_69607/1436193359.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = pd_pf.groupby('ticker').apply(lambda x: x.sort_values('Date'))


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag,returns,hi_lo_range,moving_avg_returns
4067,2020-02-19,5.18,5.240,5.08,5.18,5.18,121500.0,ALDX.csv,ALDX,2020,NaN,NaN,NaN,0.160,NaN
4068,2020-02-20,5.17,5.230,5.05,5.06,5.06,161500.0,ALDX.csv,ALDX,2020,5.18,5.18,-0.023166,0.180,NaN
4069,2020-02-21,5.07,5.100,5.03,5.05,5.05,119700.0,ALDX.csv,ALDX,2020,5.06,5.06,-0.001976,0.070,NaN
4070,2020-02-24,5.10,5.117,4.82,4.84,4.84,462600.0,ALDX.csv,ALDX,2020,5.05,5.05,-0.041584,0.297,NaN
4071,2020-02-25,4.84,4.898,4.40,4.50,4.50,341700.0,ALDX.csv,ALDX,2020,4.84,4.84,-0.070248,0.498,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229073,2015-06-26,1.62,1.690,1.55,1.64,1.64,10886800.0,UEC.csv,UEC,2015,1.60,1.60,0.025000,0.140,-0.050529
229074,2015-06-29,1.67,1.690,1.47,1.54,1.54,3615400.0,UEC.csv,UEC,2015,1.64,1.64,-0.060976,0.220,-0.053167
229075,2015-06-30,1.55,1.610,1.54,1.59,1.59,1341400.0,UEC.csv,UEC,2015,1.54,1.54,0.032468,0.070,-0.048486
229076,2015-07-01,1.62,1.690,1.45,1.48,1.48,1910200.0,UEC.csv,UEC,2015,1.59,1.59,-0.069182,0.240,-0.049950


In [ ]:
# Zhen's test
pd_pf.info()
type(pd_pf)

<class 'pandas.core.frame.DataFrame'>
Index: 332152 entries, 4067 to 229077
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Date                332152 non-null  datetime64[ns]
 1   Open                332139 non-null  float64       
 2   High                332139 non-null  float64       
 3   Low                 332139 non-null  float64       
 4   Close               332139 non-null  float64       
 5   Adj Close           332139 non-null  float64       
 6   Volume              332139 non-null  float64       
 7   source              332152 non-null  string        
 8   ticker              332152 non-null  string        
 9   Year                332152 non-null  int32         
 10  Close_lag_1         332050 non-null  float64       
 11  Adj_Close_lag       332050 non-null  float64       
 12  returns             332040 non-null  float64       
 13  hi_lo_range         332139 non-

pandas.core.frame.DataFrame

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.